<a href="https://colab.research.google.com/github/nferrucho/NPL/blob/main/curso2/ciclo5/M5U5_Automatic_Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src = "https://drive.google.com/uc?export=view&id=1Hh_G3M13P9xSNgSiQ-WnALg93XwK_hG8" alt = "Encabezado MLDS" width = "100%">  </img>

# **_Automatic Image Captioning_** (Generación Automática de Descripciones de Imágenes)
---

¿Cómo describiría lo que ve en la siguiente imagen?

<center><img src="https://drive.google.com/uc?export=view&id=1GN121S6xLJLKyLAbvvKubBLHN9sg7VR4" alt = "Ejemplo de Image Captioning con una imagen de un perro " width="70%" /></center>

Algunas personas podrían decir "Un perro jugando en la nieve", u otros podrían decir "Perro blanco con manchas marrones" y algunos otros podrían decir "Un perro en un campo abierto".

Sin duda todas estas descripciones son pertinentes para esta imagen y puede que también haya otras. Como seres humanos, nos resulta muy fácil echar un vistazo a una imagen y describirla con un lenguaje apropiado.

- ¿Cómo entrenamos un modelo para que haga esta misma tarea?


# **1. Introducción**
----
El "Automatic Image Captioning" (o Generación Automática de Descripciones de Imágenes) es un problema de aprendizaje profundo que implica **la generación automática de descripciones textuales para imágenes**. El objetivo es que un modelo pueda generar una descripción textual que resuma el contenido de una imagen en lenguaje natural, de manera similar a como lo haría un humano.

El proceso de generación automática de descripciones de imágenes se puede dividir en dos partes:

- La extracción de características de la imagen
- La generación de texto.

En la primera parte, se utiliza una red neuronal convolucional (CNN) para extraer características visuales de la imagen, mientras que, en la segunda parte, se utiliza un generador de texto, como una red neuronal recurrente (RNN) o un _Transformer_ para generar la descripción textual a partir de estas características.

La generación automática de descripciones de imágenes plantea varios desafíos.

- **En primer lugar**, la descripción generada debe ser precisa y coherente con la imagen.
- **En segundo lugar**, el modelo debe ser capaz de generar descripciones que sean gramaticalmente correctas y estén en un estilo similar al lenguaje natural.

En este notebook veremos un ejemplo de cómo modelar este problema. Usaremos el conjunto de datos [Flickr 8k](https://www.kaggle.com/datasets/adityajn105/flickr8k), una colección de referencia para la descripción y búsqueda de imágenes basada en frases, compuesta por 8000 imágenes emparejadas con cinco pies de foto diferentes que proporcionan descripciones claras de las entidades y acontecimientos más destacados. Las imágenes no suelen contener personas o lugares conocidos, sino que se seleccionaron manualmente para representar escenas y situaciones variadas.

- Como es usual, empezamos importando los paquetes necesarios:


In [ ]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
import tensorflow as tf
from tensorflow.keras.preprocessing import sequence, image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Input, layers
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Embedding, TimeDistributed, Dense,\
                                    RepeatVector, Activation, Flatten, Reshape,\
                                    concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model

In [ ]:
!git clone https://github.com/mindlab-unal/mlds5-dataset-unit5-AutomaticImageCaptioning.git

Descargamos las imágenes y las descripciones. Estas están almacenadas en un archivo de texto plano: `captions.txt`.

In [ ]:
# Definimos una función para abrir un archivo de texto:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

filename = "/content/mlds5-dataset-unit5-AutomaticImageCaptioning/captions.txt"

# Cargamos las descripciones de las imágenes
doc = load_doc(filename)

# Imprimimos los primeros 500 caracteres
print(doc[:500])

## **1.1 Preprocesamiento**
----
Como vemos, para una misma imagen tenemos cinco descripciones cortas. Vamos a procesar esta información. Creamos un diccionario llamado `descriptions` que contiene el nombre de la imagen (sin la extensión `.jpg`) como llaves, y una lista de los cinco pies de foto de la imagen correspondiente como valores.

In [ ]:
def load_descriptions(doc):
	mapping = dict()
	# Lineas a procesar
	for line in doc.split('\n'):
		# Dividir la linea según los espacios en blanco
		tokens = line.split()
		if len(line) < 2:
			continue
		# Tomar el primer token como el id de la imagen, el resto como la descripción
		image_id, image_desc = tokens[0], tokens[1:]
		# Extraer el nombre del archivo del id de la imagen
		image_id = image_id.split('.')[0]
		# Volver a convertir los tokens de la descripción en strings
		image_desc = ' '.join(image_desc)
		# Crear la lista si es necesario
		if image_id not in mapping:
			mapping[image_id] = list()
		# Almacenar la descripción
		mapping[image_id].append(image_desc)
	return mapping

# Analizar las descripciones
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))

Tenemos en total 8091 muestras para ser exactos. Veamos las descripciones de la imagen `1000268201_693b08cb0e.jpg`:

In [ ]:
descriptions['1000268201_693b08cb0e']

La imagen que describe es la siguiente:

In [ ]:
x=plt.imread('/content/mlds5-dataset-unit5-AutomaticImageCaptioning/Images/1000268201_693b08cb0e.jpg')
plt.imshow(x)

Vamos a usar un _Word Embedding_ para representar los textos y hacer el proceso final de generación. Para esto, vamos primero a normalizar los textos. Creamos una función para esto:

In [ ]:
def clean_descriptions(descriptions):
# Preparar la tabla de traducción para eliminar la puntuación
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# Tokenizar
			desc = desc.split()
			# Convertir a minúsculas
			desc = [word.lower() for word in desc]
			# Eliminar los signos de puntuación de cada palabra
			desc = [w.translate(table) for w in desc]
			# Eliminar las 's' y 'a' colgantes
			desc = [word for word in desc if len(word)>1]
			# Elimina los tokens con números
			desc = [word for word in desc if word.isalpha()]
			# Almacenar como cadena
			desc_list[i] =  ' '.join(desc)

# Limpiar descripciones
clean_descriptions(descriptions)

Limpiamos y veamos de nuevo las descripciones de la imagen `1000268201_693b08cb0e.jpg`:

In [ ]:
descriptions['1000268201_693b08cb0e']

Creamos un vocabulario de todas las palabras únicas presentes en todos los ~8000*5 (es decir, ~40000) pies de imagen (corpus) del conjunto de datos:

In [ ]:
# Convertir las descripciones cargadas en un vocabulario de palabras
def to_vocabulary(descriptions):
	# Construye una lista de todas las cadenas de descripciones
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# Resumir vocabulario
vocabulary = to_vocabulary(descriptions)
print('Original Vocabulary Size: %d' % len(vocabulary))

Esto significa que tenemos 8680 palabras únicas en los 40000 pies de foto. Escribimos todos estos pies de foto junto con los nombres de las imágenes en un nuevo archivo llamado `'descriptions.txt'` y lo guardamos. Usamos de nuevo una función:

In [ ]:
def save_descriptions(descriptions, filename):
	lines = list() # Lista vacia
	for key, desc_list in descriptions.items():
		for desc in desc_list:
			lines.append(key + ' ' + desc) # Se concatena el id de la imagen con la
																		 # descripción
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()

save_descriptions(descriptions, 'descriptions.txt')

**Particiones de entrenamiento y prueba**

Vamos a seleccionar las primeras 6000 imágenes para hacer entrenamiento; el resto será para prueba:

In [ ]:
train_names = list(descriptions.keys())[:6000]

with open(r'Flickr_8k.trainImages.txt', 'w') as fp:
    fp.write('\n'.join(train_names))

Hemos guardado los identificadores de las imágenes de entrenamiento en un archivo nuevo: `'Flickr_8k.trainImages.txt'`. Esto nos va a ser útil posteriormente durante el entrenamiento. Análogamente, guardamos los identificadores de las imágenes de prueba en el archivo `'Flickr_8k.testImages.txt'`:

In [ ]:
test_names = list(descriptions.keys())[6000:]

with open(r'Flickr_8k.testImages.txt', 'w') as fp:
    fp.write('\n'.join(test_names))

Ahora creamos una lista con la ruta de las imágenes de entrenamiento

In [ ]:
# La siguiente ruta contiene todas las imágenes
images = '/content/mlds5-dataset-unit5-AutomaticImageCaptioning/Images/'
# Crea una lista con todos los nombres de imágenes del directorio
img = glob.glob(images + '*.jpg')

# El siguiente archivo contiene los nombres de las imágenes que se utilizarán en
# los datos de entrenamiento

train_images_file = '/content/mlds5-dataset-unit5-AutomaticImageCaptioning/Flickr_8k.trainImages.txt'

# Leer de los nombres de las imágenes de entrenamiento en un conjunto
train_images = set(open(train_images_file, 'r').read().strip().split('\n'))

# Crear una lista de todas las imágenes de entrenamiento con sus nombres de ruta
# completos
train_img = []

for i in img: # img es la lista de los nombres completos de todas las imágenes
    if i[len(images):-4] in train_images: # Comprueba si la imagen pertenece al conjunto de entrenamiento
        train_img.append(i) # Añadir a la lista de imágenes de entrenamiento

Y hacemos lo mismo para la partición de prueba:

In [ ]:
# El siguiente archivo contiene los nombres de las imágenes que se utilizarán en
# los datos de prueba
test_images_file = 'Flickr_8k.testImages.txt'
test_images = set(open(test_images_file, 'r').read().strip().split('\n'))
test_img = []
for i in img:
    if i[len(images):-4] in test_images:
        test_img.append(i)

Ahora, para cada muestra juntamos las cinco descripciones en un solo texto:

In [ ]:
# Cargar descripciones limpias en memoria
def load_clean_descriptions(filename, dataset):
	# Cargar documento
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# Dividir la línea por espacios en blanco
		tokens = line.split()
		# Separar id de descripción
		image_id, image_desc = tokens[0], tokens[1:]
		# Omitir las imágenes que no están en el conjunto
		if image_id in dataset:
			# Crear lista
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# Guardar la descripción en tokens de inicio y fin
			desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			# Almacenar
			descriptions[image_id].append(desc)
	return descriptions

# Guardamos los id's en un set
train = set(list(descriptions.keys())[:6000])
len(train)
# Descripciones
train_descriptions = load_clean_descriptions('descriptions.txt', train)
print('Descriptions: train=%d' % len(train_descriptions))

> Note que hemos incluido dos tokens: `'startseq'` y `' endseq'`, que encapsulan las descripciones. Esto va a ser útil y necesario en el proceso de generación de texto, en el que vamos a usar un _Word Embedding_.

**Simplificando el vocabulario**

Si lo pensamos bien, muchas de las palabras del vocabulario aparecerán muy pocas veces, digamos 1, 2 o 3 veces. Dado que estamos creando un modelo predictivo, no nos gustaría tener todas las palabras presentes en nuestro vocabulario, sino las palabras que tienen más probabilidades de aparecer o que son comunes. Esto ayuda al modelo a ser más robusto frente a los valores atípicos y a cometer menos errores.

- Por lo tanto, sólo tenemos en cuenta las palabras que aparecen al menos 10 veces en todo el corpus. A continuación se muestra el código correspondiente:

In [ ]:
# Crear una lista de todas las descripciones
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
# Considere sólo las palabras que aparecen al menos 10 veces en el corpus
word_count_threshold = 10
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]
print('Preprocessed words %d -> %d' % (len(word_counts), len(vocab)))

Nos quedaron entonces 1652 palabras. Algo mucho más manejable que 7603.

**Índices de palabras**

Ahora, debemos tener en cuenta que las descripciones de texto son algo que queremos predecir. Entonces, durante el período de entrenamiento, los _captions_ serán las variables objetivo ($y$) que el modelo está aprendiendo a predecir. Pero la predicción de todo el pie de foto, dada la imagen, no sucede de golpe. Predeciremos el _caption_ palabra por palabra. Para esto, empezaremos por crear dos diccionarios de Python, a saber, `wordtoix` (palabra a índice) e `ixtoword` (índice a palabra).

En pocas palabras, representaremos cada palabra única en el vocabulario por un número entero (índice).

- Como se vio anteriormente, tenemos 1652 palabras únicas en el corpus y, por lo tanto, cada palabra estará representada por un índice entero entre 1 y 1652.

Estos dos diccionarios de Python se pueden usar de la siguiente manera:

- `wordtoix['abc']` -> devuelve el índice de la palabra `'abc'`.

- `ixtopalabra[k]` -> devuelve la palabra cuyo índice es `k`

El código utilizado es el siguiente:


In [ ]:
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

In [ ]:
vocab_size = len(ixtoword) + 1 # uno para los 0 añadidos
vocab_size

Hemos agregado un índice `0` que explicaremos más adelante. Mientras tanto, hay un parámetro más que necesitamos calcular: la longitud máxima de las descripciones. Lo hacemos de la siguiente manera:

In [ ]:
# Convertir un diccionario de descripciones limpias en una lista de descripciones
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# Calcula la longitud de la descripción con más palabras
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# Determinar la longitud máxima de la secuencia
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

Es decir, la descripción más larga del conjunto de entrenamiento es de 33 palabras.

## **1.2 Procesamiento de imágenes**
----
Vamos a usar una CNN para extraer representaciones de las imágenes. Vamos a tomar Inception V3, que recibe entradas de tamaño $299\times299$. Vamos entonces a definir una función para poder preproprocesar las imágenes una por una. Esto va a ser útil y necesario para el entrenamiento, pero en particular para poder ver resultados de imágenes individuales al final del proceso.


In [ ]:
def preprocess(image_path):
    # Convierte todas las imágenes a tamaño 299x299 como espera el modelo InceptionV3
    img = image.load_img(image_path, target_size=(299, 299))
    # Convertir imagen PIL a matriz numpy de 3 dimensiones
    x = image.img_to_array(img)
    # Añadir una dimensión más
    x = np.expand_dims(x, axis=0)
    # Preprocesar las imágenes usando preprocess_input() del módulo inception
    x = preprocess_input(x)
    return x

Cargamos Inception V3 con los pesos de `imagenet`

In [ ]:
model = tf.keras.applications.InceptionV3(weights='imagenet')

In [ ]:
model.summary()

Y como nos interesa la representación latente, vamos a tomar la salida de la capa `GlobalAveragePooling2D`, definiendo para esto un modelo nuevo:

In [ ]:
model_new = tf.keras.models.Model(model.input, model.layers[-2].output)

Entonces podemos definir una función que recibe una imagen y retorna el vector de características dado por InceptionV3, que tiene dimensión 2048:

In [ ]:
# Función para codificar una imagen dada en un vector de tamaño (2048, )
def encode(image):
    image = preprocess(image) # Preprocesar la imagen
    fea_vec = model_new.predict(image) # Obtener el vector de características de la imagen
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # Cambiar la forma de (1, 2048) a (2048, )
    return fea_vec

A continuación, calculamos la representación de todas las imágenes, las de entrenamiento y las de prueba. Este procedimiento puede tomar tiempo, por tanto, puede ejecutarlo, o aprovechar que las representaciones de las imágenes están guardadas en los archivos `encoded_train_images.pkl` y `encoded_test_images.pkl`.

In [ ]:
"""
# Imágenes de entrenamiento
start = time()
encoding_train = {}
for img in train_img:
    encoding_train[img[:-4]] = encode(img)
print("Time taken in seconds =", time()-start)
# Guardamos las representaciones
import pickle
with open("encoded_train_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_train, encoded_pickle)
# Imágenes de prueba
start = time()
encoding_test = {}
for img in test_img:
    encoding_test[img[:-4]] = encode(img)
print("Time taken in seconds =", time()-start)
# Guardamos las representaciones
with open("encoded_test_images.pkl", "wb") as encoded_pickle:
    pickle.dump(encoding_test, encoded_pickle)
"""

train_features = load(open("/content/mlds5-dataset-unit5-AutomaticImageCaptioning/encoded_train_images.pkl", "rb"))
print('Photos: train=%d' % len(train_features))

## **1.3. Generador de datos**
----

Este es uno de los pasos más importantes de este caso práctico. Aquí entenderemos cómo preparar los datos de forma que sea conveniente darlos como entrada al modelo. Consideremos que tenemos 3 imágenes y sus 3 leyendas correspondientes de la siguiente manera:

<center><img src="https://drive.google.com/uc?export=view&id=1qOqi5DUOPppUV_X_vdflTQs4xrrlfqBx" alt = "Matriz de resultados a partir de los indices creados para cada palabra  " width="60%" /></center>





Ahora, digamos que usamos las dos primeras imágenes y sus leyendas para entrenar el modelo y la tercera imagen para probar nuestro modelo. Las preguntas a las que hay que responder son: ¿Cómo planteamos esto como un problema de aprendizaje supervisado?, ¿Cómo es la matriz de datos?, ¿Cuántos puntos de datos tenemos?, etc.

- En **primer lugar**, tenemos que convertir las dos imágenes en sus correspondientes vectores de características de 2048 de longitud, tal y como se ha explicado anteriormente. Sean `Imagen_1` e `Imagen_2` los vectores de características de las dos primeras imágenes, respectivamente.

- En **segundo lugar**, vamos a construir el vocabulario para los dos primeros pies de foto (de entrenamiento) añadiendo los dos tokens `startseq` y `endseq` en ambos (supongamos que ya hemos realizado los pasos básicos de limpieza):

* Pie de foto 1:
```
startseq the black cat sat on grass endseq
```

* Pie de foto 2:
```
startseq the white cat is walking on road endseq
```

El vocabulario sería:
```
vocab =  {black, cat, endseq, grass, is, on, road, sat, startseq, the, walking, white}
```
Demos un índice a cada palabra del vocabulario:
```
black: 1, cat: 2, endseq: 3, grass: 4, is: 5, on: 6, road: 7, sat: 8, startseq: 9, the: 10, walking: 11, white: 12
```

Intentemos ahora plantearlo como un problema de aprendizaje supervisado en el que tenemos un conjunto de puntos de datos $D = \{X_i, y_i\}$, donde $X_i$ es el vector de características del punto de datos $i$ y $y_i$ es la variable objetivo correspondiente. Tomemos el primer vector de imágenes `Image_1` y su correspondiente leyenda `startseq the black cat sat on grass endseq`. Recordemos que el vector imagen es la entrada y el título es lo que tenemos que predecir. Pero la forma de predecir el título es la siguiente:

La primera vez, proporcionamos el vector imagen y la primera palabra como entrada e intentamos predecir la segunda palabra, es decir

* Entrada = `Imagen_1` + `'startseq'`; Salida = `'the'`

A continuación, proporcionamos el vector imagen y las dos primeras palabras como entrada e intentamos predecir la tercera palabra, es decir

* Entrada = `Imagen_1` + `'startseq the'`; Salida = `'cat'`

Y así sucesivamente... Así, podemos resumir la matriz de datos de una imagen y su correspondiente pie de foto de la siguiente manera:

<center><img src="https://drive.google.com/uc?export=view&id=1i-HVKejF8Lyp3TZciCOPe8bvZZGeMuZo" alt = "Matriz de resultados a partir del pie de foto 1 " width="100%" /></center>



Ahora debemos entender que, en cada punto de datos, no es solo la imagen la que ingresa al sistema, sino también una leyenda parcial que ayuda a predecir la siguiente palabra en la secuencia.

Dado que estamos procesando secuencias, emplearemos una red neuronal recurrente para leer estos subtítulos parciales. Sin embargo, no vamos a pasar el texto real en inglés al modelo, sino que vamos a pasar la secuencia de índices donde cada índice representa una palabra única.

- Como ya hemos creado un índice para cada palabra, ahora reemplacemos las palabras con sus índices y comprendamos cómo se verá la matriz de datos:




<center><img src="https://drive.google.com/uc?export=view&id=1IZSgSo9rFi4Yg566bkq38afdG6Al8bku" alt = "Matriz de resultados a partir de los indices creados para cada palabra  " width="100%" /></center>



Dado que estaríamos realizando un procesamiento por _batches_, debemos asegurarnos de que cada secuencia tenga la misma longitud. Por lo tanto, debemos agregar `0` (relleno con cero, ¿recuerda el índice extra?) al final de cada secuencia. Pero ¿cuántos ceros debemos agregar en cada secuencia?

Bueno, esta es la razón por la que calculamos la longitud máxima de un pie de foto, que es 33. Entonces agregaremos esa cantidad de ceros que llevará a que cada secuencia tenga una longitud de 33.

- La matriz de datos entonces se verá de la siguiente manera:


<center><img src="https://drive.google.com/uc?export=view&id=1za4j08mcNmaxpPsb6n7VsklGj2hEfIMZ" alt = "Matriz de resultados a partir de los indices creados para la longitud máxima de un pie de foto  " width="100%" /></center>




Todo este proceso esta entonces a cargo del siguiente generador de datos. Esto lo usaremos para alimentar el modelo:

In [ ]:
# Generador de datos, destinado a ser utilizado en una llamada a model.fit_generator()
def data_generator(descriptions, photos, wordtoix, max_length, num_photos_per_batch):
    X1, X2, y = list(), list(), list()
    n=0
    # Bucle para siempre sobre las imágenes
    while True:
        for key, desc_list in descriptions.items():
            n+=1
            # Recuperar la foto
            photo = photos['/content/mlds5-dataset-unit5-AutomaticImageCaptioning/Images/'+key]
            for desc in desc_list:
                # Codificar la secuencia
                seq = [wordtoix[word] for word in desc.split(' ') if word in wordtoix]
                # Dividir una secuencia en varios pares X, y
                for i in range(1, len(seq)):
                    # Dividir en pares de entrada y salida
                    in_seq, out_seq = seq[:i], seq[i]
                    # Rellenar secuencia de entrada
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # Codificar la secuencia de salida
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # Almacenar
                    X1.append(photo)
                    X2.append(in_seq)
                    y.append(out_seq)
            # Producir los datos del lote
            if n==num_photos_per_batch:
                yield [[array(X1), array(X2)], array(y)]
                X1, X2, y = list(), list(), list()
                n=0

## **1.4 _Word Embedding_**
---
Para representar el texto, vamos a utilizar [**Glove**](https://nlp.stanford.edu/projects/glove/), un método de embedding que permite capturar y representar las características esenciales de las palabras y su contexto.
- Glove, que es un acrónimo de "Global Vectors for Word Representation", es un modelo de lenguaje diseñado para transformar el texto en vectores de alta dimensión, logrando una representación más rica y precisa.

Este enfoque de embedding considera las palabras como objetos globales en un espacio vectorial, donde las relaciones semánticas y gramaticales entre ellas se reflejan en la cercanía y orientación de los vectores correspondientes. En Glove, se presta especial atención a la incorporación de información contextual y la captura de relaciones semánticas a nivel global. Esto permite que el modelo identifique patrones y estructuras en el texto de manera más efectiva que otros enfoques de embedding, lo que resulta en una mejor comprensión del contenido y una mayor capacidad para abordar tareas complejas relacionadas con el lenguaje.

- Para utilizar Glove en Keras, primero debemos obtener los embeddings pre-entrenados de Glove y luego cargarlos en una capa de Embedding de Keras. Vamos a usar una representación de 200 dimensiones.

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

Una capa de Embedding en Keras es una capa especializada que transforma representaciones de palabras basadas en enteros (índices) en representaciones de vectores densos (embeddings).

In [ ]:
# Cargar los vectores de Glove
embeddings_index = {} # diccionario vacío
f = open('glove.6B.200d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

GloVe contiene la representación para 400000 palabras. Vamos a seleccionar solo las que nos interesan según nuestro vocabulario:

In [ ]:
embedding_dim = 200
# Obtener un vector denso de 200 dim para cada una de las 1653 palabras de nuestro vocabulario
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in wordtoix.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Las palabras no encontradas en el índice de incrustación serán todos ceros
        embedding_matrix[i] = embedding_vector

Veamos

In [ ]:
embedding_matrix.shape

`embedding_matrix` contiene la representación de todas las palabras de nuestro vocabulario. Esta matriz constituirá el conjunto de pesos de la capa `Embedding` de nuestro modelo generador de descripciones.

# **2. Descripción Automática de Imágenes**
----

Ahora sí, vamos a crear nuestro modelo de _automatic image captioning_.

<center><img src="https://drive.google.com/uc?export=view&id=1Oz3GJVWnAYH7Dl9dcYWEnALh1_amq76S" alt = "Matriz de resultados a partir del pie de foto 1 " width="80%" /></center>


## **2.1 Definición del modelo**
----

**_Embedding Layer_**

Una capa de `Embedding` en Keras es una capa especializada que transforma representaciones de palabras basadas en enteros (índices) en representaciones de vectores densos (_embeddings_). Al crear una capa de `Embedding`, es necesario especificar dos parámetros principales:

*        `input_dim`: El tamaño del vocabulario, es decir, el número total de palabras únicas en el conjunto de datos, en nuestro caso 1653.
*        `output_dim`: La dimensión del espacio vectorial en el que se representarán las palabras (también conocida como la dimensión del embedding); en nuestro caso, 200.

La capa de `Embedding` se incluye en la arquitectura de la red neuronal como una de las primeras capas. Esta recibe los índices de las palabras de la secuencia de texto, y genera una representación global a partir de las representaciones individuales de las palabras. Durante el entrenamiento, la capa de `Embedding` ajusta los pesos de los vectores de las palabras para minimizar el error en la tarea específica (por ejemplo, clasificación de texto o generación de texto).

Después del entrenamiento, la capa de `Embedding` puede utilizarse para transformar las palabras en sus representaciones vectoriales correspondientes, que luego se utilizan como entrada para las capas posteriores de la red neuronal. Esto es lo que nosotros haremos.

- **No queremos entrenar** la capa `Embedding`, porque ya tenemos la representación que nos ofrece **Glove**. Solo vamos a usar la capa como un mecanismo de codificación del texto.

Definimos la arquitectura del modelo:

In [ ]:
inputs1 = Input(shape=(2048,))
fe1 = Dropout(0.5)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)
inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, embedding_dim, mask_zero=True)(inputs2)
se2 = Dropout(0.5)(se1)
se3 = LSTM(256)(se2)
decoder1 = tf.keras.layers.Add()([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)
model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.summary()

Identifiquemos la capa de `Embedding`

In [ ]:
model.layers[2]

Como mencionamos anteriormente, `embedding_matrix` contiene la representación de todas las palabras de nuestro vocabulario (según GloVe). Debemos entonces cargar esta matriz de pesos a la capa `Embedding` de nuestro modelo generador de descripciones, y además congelamos la capa, para que no se alteren estas representaciones.  

In [ ]:
model.layers[2].set_weights([embedding_matrix])
model.layers[2].trainable = False

Veamos el diagrama del modelo:

In [ ]:
plot_model(model, show_shapes=True,)

**Entendamos el modelo**

El modelo recibe dos entradas. En la gráfica, vemos dos ramas.

- La parte izquierda corresponde a la entrada de la secuencia de texto. Es de tamaño 33 porque 33 es la longitud de la secuencia más larga en el conjunto de entrenamiento. El texto se codifica con la capa `Embedding`, pasa por una capa de `dropout` y luego por una capa LSTM (que estudiamos en la Unidad 4 de este módulo).

- Por la rama de la derecha entra la imagen, codificada ya como un vector de 2048 dimensiones (gracias a InceptionV3). Esta representación pasa por una capa de `dropout` y luego por una capa densa para obtener una representación de 256 dimensiones.

Las representaciones de texto e imagen se suman en la capa `add`, obteniendo una representación conjunta de imagen y texto, que entra entonces a un modelo decodificador de dos capas densas, cuya salida indica la probabilidad de ocurrencia de la siguiente palabra de la secuencia de texto.


## **2.2 Compilación y entrenamiento**
----

Compilamos usando `categorical_crossentropy` y `adam` como optimizador, con la tasa de aprendizaje por defecto (0.001):

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

Como primera instancia, vamos a entrenar durante 20 epochs. Luego vamos a bajar la tasa de aprendizaje a 0.0001 para ajustar un poco más el modelo durante 10 epochs. Durante el entrenamiento se guardarán los pesos después de cada epoch.

> **Nota**: este entrenamiento puede demorarse más de una hora usando una GPU estándar de Colab. Por practicidad, podemos cargar los pesos obtenidos en un entrenamiento anterior.


In [ ]:
"""
epochs = 20
number_pics_per_bath = 3
steps = len(train_descriptions)//number_pics_per_bath

for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('./model_weights/model_' + str(i) + '.weights.h5')

model.optimizer.lr = 0.0001
epochs = 10
number_pics_per_bath = 6
steps = len(train_descriptions)//number_pics_per_bath

for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, wordtoix, max_length, number_pics_per_bath)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

model.save_weights('./model_weights/model_30.weights.h5')
"""
# Cargando los pesos de un entrenamiento anterior
# En caso de entrenar por tu cuenta usar .weights.h5
model.load_weights('/content/mlds5-dataset-unit5-AutomaticImageCaptioning/model_30.h5')

## **2.3 Visualización de resultados**
----

Durante inferencia, el modelo recibe como entrada la imagen, y el token `startseq`. Este es el punto de partida. En la primera iteración el modelo predice la siguiente palabra de la secuencia, la de mayor probabilidad en el vocabulario de 1653 elementos. Supongamos que la siguiente palabra es: `The`. Entonces en la siguiente iteración el modelo recibe la imagen y la secuencia `startseq The`, y calculará la siguiente palabra. El proceso sigue hasta que en un momento el siguiente token en la secuencia sea `endseq`.

- Carguemos las representaciones de las imágenes del conjunto de prueba:


In [ ]:
with open("/content/mlds5-dataset-unit5-AutomaticImageCaptioning/encoded_test_images.pkl", "rb") as encoded_pickle:
    encoding_test = load(encoded_pickle)

Y definimos la función `captioning`. Esta función recibe la representación de la imagen de prueba, e inicial la generación iterativa de texto con `startseq`:

In [ ]:
def captioning(photo):
    in_text = 'startseq'
    for i in range(max_length):
        sequence = [wordtoix[w] for w in in_text.split() if w in wordtoix]
        sequence = pad_sequences([sequence], maxlen=max_length, padding='post')
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = ixtoword[yhat]
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

El siguiente código nos permite visualizar uno a uno los resultados:

In [ ]:
z = 0

Cada vez que ejecute la siguiente celda, la imagen y la descripción generada por nuestro modelo va a cambiar:

In [ ]:
z+=1
pic = list(encoding_test.keys())[z].split('/')[-1]
image = encoding_test[list(encoding_test.keys())[z]].reshape((1,2048))
x=plt.imread('/content/mlds5-dataset-unit5-AutomaticImageCaptioning/Images/'+pic+'.jpg')
plt.imshow(x)
plt.show()
print("Caption:",captioning(image))

¡Hemos terminado, buen trabajo!

# **Recursos adicionales**
---


- [*Image Captioning with Keras*](https://towardsdatascience.com/image-captioning-with-keras-teaching-computers-to-describe-pictures-c88a46a311b8)

- [*Flickr 8k Dataset*](https://www.kaggle.com/datasets/adityajn105/flickr8k)

* _Origen de los íconos_

    - Yadav,A. Vishwakarma,A. Panickar,S. Kuchiwale,S (2020). IRJET: Real Time Video to Text Summarization using Neural Network - CNN + LSTM for Image Captioning [Imagen] https://www.irjet.net/archives/V7/i12/IRJET-V7I12333.pdf
    - Limited, A. (s. f.). A black cat with a white front patch sat on the grass. Alamy images. https://www.alamy.com/stock-photo-a-black-cat-with-a-white-front-patch-sat-on-the-grass-36422236.html
    - The white cat crosses the road. The concept will be: the white cat crossing the road brings good luck.   foto de Stock. (s. f.). Adobe Stock. https://stock.adobe.com/es/images/the-white-cat-crosses-the-road-the-concept-will-be-the-white-cat-crossing-the-road-brings-good-luck/288570621
    - Limited, A. (s. f.-b). Black cat walking on grass in the garden. Alamy images. https://www.alamy.com/black-cat-walking-on-grass-in-the-garden-image63426915.html



# **Créditos**
---

* **Profesor:** [Fabio Augusto Gonzalez](https://dis.unal.edu.co/~fgonza/)
* **Asistentes docentes :**
  * [Santiago Toledo Cortés](https://sites.google.com/unal.edu.co/santiagotoledo-cortes/)
* **Diseño de imágenes:**
    - [Mario Andres Rodriguez Triana](mailto:mrodrigueztr@unal.edu.co).
* **Coordinador de virtualización:**
    - [Edder Hernández Forero](https://www.linkedin.com/in/edder-hernandez-forero-28aa8b207/).

**Universidad Nacional de Colombia** - *Facultad de Ingeniería*